
# ΕΡΓΑΣΙΑ "ΑΝΑΛΥΣΗ ΚΑΙ ΔΙΑΧΕΙΡΙΣΗ ΧΑΡΤΟΦΥΛΑΚΙΟΥ"
### ΚΟΥΚΟΥΛΑΡΗΣ ΝΙΚΟΛΑΟΣ (7190300) 
### ΤΜΗΜΑ ΛΟΓΙΣΤΙΚΗΣ ΚΑΙ ΧΡΗΜΑΤΟΟΙΚΟΝΟΜΙΚΗΣ (ΟΙΚΟΝΟΜΙΚΟ ΠΑΝΕΠΙΣΤΗΜΙΟ ΑΘΗΝΩΝ)

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from functools import reduce
import numpy as np

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

from pypfopt import HRPOpt

from pypfopt.efficient_frontier import EfficientCVaR

## Άντληση δεδομένων απο το API του YahooFinance.com
Συγκεκτιμένα αντλούμε δεδομένα κλεισίματος (Close) απο την ημερομηνία 1/1/2017 (start) εως αρχες του έτους 2020 (end) και αποθηκεύμε στη μεταβλητή data. Στην συνέχεια ορίζουμε μια συνάρτηση (combine stocks) με απότερο σκοπό να συνδιάσουμε τα χρεόγραφα μας βάση των tickers τους και στο τέλος επιστρέφει το (df merged) όπου κατανέμει τα δεδομένα όπου αντλήσαμε. Τέλος, αποθηκεύουμε σε μια λίστα (stocks) όλα τα σύμβολα (tickers) των μετοχών που θα πραγματοποιήσουμε την ανάλυση. 
Η λίστα portofolio εκτελεί τη συνάρτιση get_stock η οποία πέρνει ως όρισμα τη λίστα stocks.

In [2]:

start = dt.datetime(2019,1,1)
end = dt.datetime(2021,1,1)

def get_stock(ticker):
    data = web.DataReader(ticker,"yahoo",start,end)
    data[ticker] = data["Close"]
    data = data[[ticker]] 
    return data 


def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    return df_merged


stocks = ["BTC-USD", "XRP-USD", "ETH-USD", "GOOGL", 
          "FB", "AAPL", "COST", "WMT", "KR", "JPM", 
          "BAC", "HSBC"]
portfolio = combine_stocks(stocks)

portfolio = portfolio.dropna()
portfolio

PORTFOLIO_CAPITAL = 100000



### Αποθηκεύουμε το portfolio σε ενα αρχείο csv ,με μόνη διαφορά οτι θέλουμε να αφαιρέσει τις ημερομηνίες (index) 
Ο λόγος για τον οποίο αποθήκεύουμε σε csv είναι διότι δεν θέλουμε να ζητάμε τα 
δεδομένα απο το API του YahooFinance, καθώς αυτό είναι χρονοβόρο 

In [3]:
portfolio.to_csv("portfolio.csv", index=False)


In [4]:
portfolio = pd.read_csv("portfolio.csv")
portfolio

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,3943.409424,0.375243,155.047684,1054.680054,135.679993,39.480000,204.759995,93.339996,27.299999,99.309998,24.959999,40.880001
1,3836.741211,0.360224,149.135010,1025.469971,131.740005,35.547501,200.419998,92.860001,27.350000,97.110001,24.559999,40.430000
2,3857.717529,0.356747,154.581940,1078.069946,137.949997,37.064999,206.240005,93.440002,27.660000,100.690002,25.580000,41.610001
3,4025.248291,0.364347,151.699219,1075.920044,138.050003,36.982498,207.000000,94.540001,27.920000,100.760002,25.559999,41.049999
4,4030.847900,0.365315,150.359634,1085.369995,142.529999,37.687500,208.550003,95.199997,28.459999,100.570000,25.510000,41.160000
...,...,...,...,...,...,...,...,...,...,...,...,...
500,23735.949219,0.337819,611.607178,1734.160034,267.399994,131.970001,364.579987,143.500000,31.530001,124.519997,29.959999,26.049999
501,27084.808594,0.247958,730.397339,1773.959961,277.000000,136.690002,371.059998,145.220001,31.389999,125.339996,30.129999,25.850000
502,27362.437500,0.220962,731.520142,1757.760010,276.779999,134.869995,372.720001,144.300003,31.459999,125.010002,30.010000,25.820000
503,28840.953125,0.211828,751.618958,1736.250000,271.869995,133.720001,374.450012,144.179993,31.549999,125.360001,29.980000,25.940001


## 1. Mean Variance optimization
H πρώτη μέθοδος που θα προσεγκίσουμε είναι μέθοδος "Βελτιστοποίησης Μέσου-Διακύμανσης" που χρησιμοποιούμε το υπόδειγμα του Harry Markowitz για την οποία θα χρειαστόυμε να υπολογίσουμε το ιστορικό μέσο (mean) των αποδόσεων των μετοχών που απαρτίζεται το χαρτοφυλάκιο και την συνδιακύμανση αυτών (covariance matrix) αποθηκεύοντας τα στα data frames mu και S αντίστοιχα.

In [5]:
mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()
S

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
BTC-USD,0.482934,0.336800,0.464490,0.041323,0.058269,0.050997,0.025331,0.025234,0.007288,0.048176,0.054841,0.028005
XRP-USD,0.336800,1.091468,0.554475,0.041594,0.058228,0.053748,0.030051,0.031010,0.017679,0.065698,0.075340,0.033597
ETH-USD,0.464490,0.554475,0.714506,0.065417,0.075877,0.081658,0.040160,0.045833,0.015239,0.069996,0.082158,0.034261
GOOGL,0.041323,0.041594,0.065417,0.118039,0.081563,0.077430,0.038301,0.030299,0.016408,0.065383,0.072519,0.041883
FB,0.058269,0.058228,0.075877,0.081563,0.156051,0.090818,0.039465,0.029238,0.017458,0.060798,0.068535,0.036890
AAPL,0.050997,0.053748,0.081658,0.077430,0.090818,0.155593,0.048737,0.038845,0.021533,0.072513,0.081954,0.045780
COST,0.025331,0.030051,0.040160,0.038301,0.039465,0.048737,0.075455,0.037061,0.025146,0.033472,0.037168,0.019852
WMT,0.025234,0.031010,0.045833,0.030299,0.029238,0.038845,0.037061,0.079811,0.029391,0.029411,0.033434,0.015537
KR,0.007288,0.017679,0.015239,0.016408,0.017458,0.021533,0.025146,0.029391,0.108634,0.012540,0.018589,0.006043
JPM,0.048176,0.065698,0.069996,0.065383,0.060798,0.072513,0.033472,0.029411,0.012540,0.176309,0.155944,0.081119


In [6]:
mu

BTC-USD    1.711914
XRP-USD   -0.234574
ETH-USD    1.181412
GOOGL      0.289098
FB         0.418896
AAPL       0.833287
COST       0.356505
WMT        0.242720
KR         0.078596
JPM        0.131163
BAC        0.101972
HSBC      -0.203881
dtype: float64

## 1.1 Αποτελεσματικό Σύνορο και Ποσοστά επένδυσης σε κάθε χρεόγραφο (W1-Wn)
Με την χρήση της συνάρτησης EfficientFrontier που πέρνει ως όρισμα τα mu και S (τα οποία υπολογίσαμε νωρίτερα ) μας δίνεται η δυνατότητα να υπολογίσουμε το αποτελεσματικό σύνορο χαρτοφυλακίων και στη συνέχεια τα w (weights) ,δηλαδή το ποσοστό (%) των διαθέσιμων κερφαλαίων που πρέπει να επενδύσουμε στην κάθε μετοχή κεχωριστά για να αποκαλείται Αποτελεσματικό Χαρτοφυλλάκιο. Στη συνέχεια εκτυπώνουμε τα αποτελέσματ των συναρτήσεων.

In [7]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights_mvo = ef.clean_weights()
print(dict(cleaned_weights_mvo))
ef

{'BTC-USD': 0.38579, 'XRP-USD': 0.0, 'ETH-USD': 0.0, 'GOOGL': 0.0, 'FB': 0.0, 'AAPL': 0.50178, 'COST': 0.11243, 'WMT': 0.0, 'KR': 0.0, 'JPM': 0.0, 'BAC': 0.0, 'HSBC': 0.0}


## 1.2 Απόδοση Χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "mvo_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή με αυτό το χαρτοφυλάκιο, ομοίως στη "mvo_annual_volatility" τη ετήσια μεταβλητότητα - όγκο των μετοχών που περιέχει το χαρτοφυλάκιο μας και τέλος για την ευρεση του Sharpe Ratio αλλα και τον υπολογισμό του ,το αποθηκεύουμε στο "mvo_sharpe_r".Εκτελόντας αυτή τη γραμμή κώδικα θα μας επιστρέψει τα αποτελέσματα που ζητήσαμε.

In [8]:
mvo_exp_annual_ret, mvo_annual_volatility , mvo_sharpe_r = ef.portfolio_performance(verbose=True)

Expected annual return: 111.9%
Annual volatility: 37.3%
Sharpe Ratio: 2.94


## 1.3 Add Funds (How many stocks (n) according to the Funds)
Αφού πριν βρήκαμε τα ποσoτά των κεφαλαίων του κατόχου χαρτοφυλακίου που πρέπει να επενδυθούν (w), τώρα μπρορούμε να βρούμε τα χρήματα (σε δολάρια) που πρέπει να επενδύσουμε με γνώμονα οτι ο επενδυτής έχει στη διακριτική του ευκαίρια το ποσό των 100.000$. Για να πραγματοποιηθεί αυτό φτιάχνουμε μια μεταβλητή allocation που στη συνέχεια την εκτυπώνουμε  ,δηλαδή διανομή των χρημάτων ως που να είναι αποτελεσματική και κερδοφόρα η τοποθετησή τους σε ενα αξιόγραφο του χαρτοφυλακίου και στο τέλος εκτυπώνει τα υπολοιπόμενα λεφτά του επενδυτή που παρέμηναν εκτός λόγω του οτι δεν απαιτούνται για τη δημιουργία αποτελεσματικού χαρτοφυλακίου.

In [9]:
latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'AAPL': 378, 'BTC-USD': 1, 'COST': 29}
Funds remaining: $9914.84


## 2. Hierarchical Risk Parity
Στη δεύτερη μέθοδο όπου θα κάνουμε ανάλυση δε απαιτείται ο υπολογισμός του πίνακα συνδιακίμανσης όπως κάναμε στη πρώτη μέθοδο. Για αρχή θα εκτυπώσουμε τις αποδόσεις των χρεογράφων του χαρτοφυλλακίου με τις οποίες θα ασχοληθούμε στη συνέχεια.

In [10]:
returns = portfolio.pct_change().dropna()
returns

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
1,-0.027050,-0.040025,-0.038135,-0.027696,-0.029039,-0.099607,-0.021196,-0.005142,0.001832,-0.022153,-0.016026,-0.011008
2,0.005467,-0.009652,0.036523,0.051294,0.047138,0.042689,0.029039,0.006246,0.011335,0.036865,0.041531,0.029186
3,0.043427,0.021304,-0.018648,-0.001994,0.000725,-0.002226,0.003685,0.011772,0.009400,0.000695,-0.000782,-0.013458
4,0.001391,0.002657,-0.008831,0.008783,0.032452,0.019063,0.007488,0.006981,0.019341,-0.001886,-0.001956,0.002680
5,0.001104,0.015261,0.002949,-0.003427,0.011927,0.016982,0.006473,-0.003256,-0.014406,-0.001690,0.009800,0.006803
...,...,...,...,...,...,...,...,...,...,...,...,...
500,0.021281,0.306328,0.047785,0.003431,-0.002648,0.007712,0.007433,0.001955,0.004460,-0.004398,-0.002995,0.000768
501,0.141088,-0.266003,0.194226,0.022951,0.035901,0.035766,0.017774,0.011986,-0.004440,0.006585,0.005674,-0.007677
502,0.010250,-0.108873,0.001537,-0.009132,-0.000794,-0.013315,0.004474,-0.006335,0.002230,-0.002633,-0.003983,-0.001161
503,0.054034,-0.041337,0.027475,-0.012237,-0.017740,-0.008527,0.004642,-0.000832,0.002861,0.002800,-0.001000,0.004648


## W1-Wn
Στη πρώτη μέθοδο χρειάστεικε η εύρεση του Αποτελεσματικού Συνόρου των χαρτοφυλλακίων ,με τη παρούσα μέθοδο θα βρούμε τα ποσοστά (w) που πρέπει να επενδυθούν με την χρήση του αλγορίθμου optimize και στη συνέχεια τα εκτυπώνουμε.

In [11]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()
hrp_weights

OrderedDict([('AAPL', 0.07095950295960557),
             ('BAC', 0.06924089146199487),
             ('BTC-USD', 0.018407465488070037),
             ('COST', 0.14594549821896488),
             ('ETH-USD', 0.012226480053868156),
             ('FB', 0.04094273859146721),
             ('GOOGL', 0.057646220041030045),
             ('HSBC', 0.12880183072187204),
             ('JPM', 0.07632930470836781),
             ('KR', 0.23082115405810494),
             ('WMT', 0.1343747840819969),
             ('XRP-USD', 0.014304129614657511)])

## 2.1 Aπόδοση του χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "hrp_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή με αυτό το χαρτοφυλλάκιο, ομοίως στη "hrp_annual_volatility" τη ετήσια μεταβλητότητα - όγκο των μετοχών που περιέχει το χαρτοφυλλακιό μας και τέλος για την ευρεση του Sharpe Ratio αλλα και τον υπολογισμό του ,το αποθηκεύουμε στο "hrp_sharpe_r".Εκτελόντας αυτή τη γραμμή κώδικα θα μας επιστρέψει τα αποτελέσματα που ζητήσαμε και ξανάεκτυπώνουμε τα απαιτούμενα weights ξανά για λόγους υπενθύμισης.

In [12]:
hrp_exp_annual_ret, hrp_annual_volatility, hrp_sharpe_r = hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 24.0%
Annual volatility: 21.6%
Sharpe Ratio: 1.02
{'AAPL': 0.07095950295960557, 'BAC': 0.06924089146199487, 'BTC-USD': 0.018407465488070037, 'COST': 0.14594549821896488, 'ETH-USD': 0.012226480053868156, 'FB': 0.04094273859146721, 'GOOGL': 0.057646220041030045, 'HSBC': 0.12880183072187204, 'JPM': 0.07632930470836781, 'KR': 0.23082115405810494, 'WMT': 0.1343747840819969, 'XRP-USD': 0.014304129614657511}


## 2.2 Add Funds (How many stocks (n) according to the Funds)
Χωρίς να χρησιμοποιήσουμε τις τρέχουσες τιμές όπως στη πρώτη μέθοδο ,μπρορούμε να βρούμε τα χρήματα (σε δολάρια) που πρέπει να επενδύσουμε με γνώμονα οτι ο επενδυτής έχει στη κατοχή του το ποσό των 100.000$. Για να πραγματοποιηθεί αυτό φτιάχνουμε μια μεταβλητή allocation που στη συνέχεια την εκτυπώνουμε  ,δηλαδή διανομή των χρημάτων ως που να είναι αποτελεσματική και κερδοφόρα η τοποθετησή τους σε ενα αξιόγραφο του χαρτοφυλλακίου και στο τέλος εκτυπώνει τα υπολοιπόμενα λεφτά (leftover) του επενδυτή που παρέμηναν εκτός λόγω του οτι δεν απαιτούνται για τη δημιουργία αποτελεσματικού χαρτοφυλλακίου.

In [13]:
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da_hrp.greedy_portfolio()
print("Discrete allocation (HRP):", allocation)
print("Funds remaining (HRP): ${:.2f}".format(leftover))


Discrete allocation (HRP): {'KR': 726, 'COST': 39, 'WMT': 93, 'HSBC': 497, 'JPM': 60, 'AAPL': 54, 'BAC': 228, 'GOOGL': 4, 'FB': 15, 'XRP-USD': 6506, 'ETH-USD': 2}
Funds remaining (HRP): $250.58


## 3. Μέσος λαμβάνοντας υπόψη το Value at Risk (VaR)
Στη τρίτη και τελευταία μέθοδο που θα χρησιμοποιήσουμε πραγματοποιήται μια εκτίμηση στο χειρότερο σενάριο ο επενδυτής να έχει loss(έλλειμα) επενδύοντας σε αυτό το χαρτοφυλλάκιο. Δηλαδή δίνει την απάντηση στο εύλογο ερώτημα με ποία μετοχή θα χάσει τα περισσότερα χρήματα(πσοστά κεφαλαίων). Είναι μια εναλλακτική μορφή της πρώτης μεθόδου (Μεσου-Διακύμανσης). Επιστρέφουμε πάλι τα weights  και τα εκτυπώνουμε μέσω της χρήσης του αλγορίθμου το EfficientCVAR που πέρνει ως όρισμα τα mu και S απο την πρώτη μέθοδο.

In [14]:
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

{'BTC-USD': 0.0, 'XRP-USD': 0.0, 'ETH-USD': 0.04604, 'GOOGL': 0.01939, 'FB': 0.0, 'AAPL': 0.0, 'COST': 0.0, 'WMT': 0.0, 'KR': 0.0, 'JPM': 0.93457, 'BAC': 0.0, 'HSBC': 0.0}


## Aπόδοση Χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "cvar_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή που κατέχει αυτό το χαρτοφυλλάκιο, ομοίως στη "cvar_value_at_risk " τον κίνδυνο που ενέχει το χαρτοφυλλάκιο να μην επιστέψει τις αναμενόμενες επιστροφές στον επενδυτή. Δηλαδή ,μετράει τη φερεγγυότητα του χαρτοφυλλακίου.Με αυτή τη μέθοδο δεν υπολογίζουμε ούτε Sharpe Ratio αλλά ούτε το Volatility και τα αντικαστήσαμε με τον υπολοφισμό του VaR.

In [15]:
cvar_exp_annual_ret, cvar_value_at_risk = ef_cvar.portfolio_performance(verbose=True)

Expected annual return: 18.3%
Conditional Value at Risk: -46.72%


## 3.2 Add Funds (How many stocks (n) according to the Funds)
Με γνώμονα ότι ο επενδυτής διαθέτει το ποσό των 100.000$ ,πρέπει να διαμοιράσει τα κεφάλαια του όπου κρίνεται αποδοτικό, κερδοφόρο και να επιστρέψει πόσο απο το χρηματικό ποσό θα παραμείνει εκτός επένδυσης. Η συνάρτιση Allocation και leftover θα μας εκτυπώσουν αυτά τα αποτελέσματα.



In [16]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))

Discrete allocation (CVAR): {'JPM': 735, 'ETH-USD': 6, 'GOOGL': 1, 'HSBC': 1, 'BAC': 1, 'KR': 1, 'XRP-USD': 1, 'AAPL': 1, 'WMT': 1}
Funds remaining (CVAR): $59.05


## Real analysis of 2021, using the above methods
Παρακάτω δοκιμάζουμε τις τρείς μεθόδους αν πράγματι επιστρέφουν φερέγγυες τιμές. Απο οτι παρατηρούμε η πρώτη μέθοδος δεν είναι ακριβής σε σύγκριση με τις άλλες δύο.

In [17]:
start = dt.datetime(2021,1,1)
end = dt.datetime(2021,12,25)



In [18]:
# weights_dict = dict(cleaned_weights)
weights_dict = dict(cleaned_weights_mvo)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using Mean Variance method: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))

True returns of 2021 using Mean Variance method: 51.36%


In [19]:

# weights_dict = dict(cleaned_weights)
weights_dict = dict(hrp_weights)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using HRP: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))
    

True returns of 2021 using HRP: 39.81%


In [20]:

weights_dict = dict(cleaned_weights)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using cVar: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))

True returns of 2021 using cVar: 45.70%
